In [1]:
from theano.sandbox import cuda
cuda.use('gpu2')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): Ignoring call to use(2), GPU number 0 is already in use.


In [2]:
%matplotlib inline
import utils #; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

## Setup

In [46]:
#path = get_file('Hofmannsthal.txt', origin="http://www.gutenberg.org/files/31625/31625-0.txt")
path = 'Hofmannsthal.txt'
text = open(path).read()
print('corpus length:', len(text))

corpus length: 60601


In [47]:
!head {path} -n1000 | tail -n25

In [101]:
#path = 'data/wiki/'
#text = open(path+'small.txt').read().lower()
#print('corpus length:', len(text))

#text = text[0:1000000]

corpus length: 137587200


In [48]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 79


In [49]:
chars.insert(0, "\0")

In [50]:
''.join(chars[1:-6])

"\n\r !'(),-.:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz\x80\x84\x96\x9c\x9f\xa4"

In [51]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [52]:
idx = [char_indices[c] for c in text]

In [53]:
idx[:10]

[36, 29, 32, 20, 32, 77, 70, 22, 26, 23]

In [34]:
''.join(indices_char[i] for i in idx[:70])

'vorfr\xc3\x9chling\r\n\r\n\r\nes l\xc3\xa4uft der fr\xc3\xbchlingswind\r\ndurch kahle alleen,\r\ns'

## Preprocess and create model

In [54]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 60562


In [55]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [56]:
sentences.shape, next_chars.shape

((60560, 40), (60560, 40))

In [57]:
n_fac = 24

In [58]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])    

In [59]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

## Train

In [63]:
def print_example():
    seed_string="""Es läuft der Frühlingswind
Durch kahle"""
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [61]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
60560/60560 [==============================] - 148s - loss: 2.2662     

In [64]:
print_example()

Es läuft der Frühlingswind
Durch kahler so litauhen
Und schlauert ohfere KgehKgegragt, und meine
Ind kilnes Sterbend spielen Meinen,
Das anderst Geellamten hit.

Schnal!

Und wieder mich schweiken aus der Schwandlicht,
Mir allelnd ons sitr swieg gerrangsnhanges
Das freure Rit!

Megrt
Stäßt in meines Tfipm versieft, am wie er schein,
Werwamll


In [65]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
60560/60560 [==============================] - 148s - loss: 1.5412     

In [66]:
print_example()

Es läuft der Frühlingswind
Durch kahlen läufe,
Durchschliegen bescheine Wären süß
Und änher weg und meine Stimme, Geschick
Mit dünkt:
Sich hatt des Schwebend- Schon und meinem Glast!

In das über Fücke lieben sich des leelen schönhte Fürsch bipd
Und kreine Bäde ich schon eander sehr bekraum auf fahlen Augen
Verwandt er sich, mich herste 


In [67]:
model.optimizer.lr=0.001

In [68]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
60560/60560 [==============================] - 148s - loss: 1.2068     

In [69]:
print_example()

Es läuft der Frühlingswind
Durch kahle Augen,
Srelbimmen Meeres nicht und rammt
Und über sich mit herzten Wolfen
Spielen jeder König
Des Heimkeh! Wenn ich in entgegen,
Ein Weib, das ist der Haffen, Mwieget röniglicher Nachen
Der Wohnung sind an diesen kleinen. Esle

Füßtig ungewändigt.

 (=So Ein Gerühren: Spielen!
Auf dem botten Flumen w


In [70]:
model.optimizer.lr=0.0001

In [71]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
60560/60560 [==============================] - 148s - loss: 0.9950     

In [72]:
print_example()

Es läuft der Frühlingswind
Durch kahle Alleen
Vergessen kannst --,
Den ich jann und Herr verschlagenen
 Reit, das einer ahso gleicht nun nacht.

Er krocht und sied -- wovich lier sein eignes Haar.

Doch gleich daren nachlen oder auf dem Döden zu,
Wo seine Honde aus den Wein zu bast,
Und alle =genn hab ich keun gekohltem Frauen
Des Fodes -- Aug d


In [73]:
model.save_weights('data/char_rnn_Hofmann.h5')

In [74]:
model.optimizer.lr=0.00001

In [75]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
60560/60560 [==============================] - 148s - loss: 0.8661     

In [76]:
print_example()

Es läuft der Frühlingswind
Durch kahle Alleen,
Seltsame Dinge hürsche Dinge hören,
Bleibt immerhin, wis ich mich haben
Dämonisch gelleider, Blit von dir sitzest,
In demsen Lilt der Hand.
Den Boden winder, und jo Häusern Wohnen,
Wie von inn Haar und Khin,
Das kedes auf wie die Seele des gewalten.

So denkt ihn! Wir stand und sterben schwer zur 


In [77]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
60560/60560 [==============================] - 148s - loss: 0.7841     

In [78]:
print_example()

Es läuft der Frühlingswind
Durch kahle Flätte
Aussenden, daß den Weinsam sind
Und wie mein eignes Haar,

So eins mit der Guntel der Frau zeichnen sich hürs Leben nichts
Und wechselndem Erlebnes Gefährtin,
Als hätte dieser nähme zien,
Du kennst, mich fürs Leben auch --
›Kind, weil sie sich um,
Die Hyazinthen das Haus, im Hintergrunde niema


In [79]:
print_example()

Es läuft der Frühlingswind
Durch kahle Alleen
Seltsame Dinge sind
In seinem Beib war so begabt,
Ind dieses Kind
Mir wie die Augen, denkt zurückgar sich im Mtuegend
Bei den Hauppegleiten
Wie jetzt für das Anschaun von deinem Land
Weit Bote aller Boten, namenlos
Und Bäum' und Gras, zu ihren Füßen
Der Netens wär kein ›Dies ist vergahden, darf


In [80]:
model.save_weights('data/char_rnn_Hofmann.h5')

In [81]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=4)

Epoch 1/4
60560/60560 [==============================] - 148s - loss: 0.7268     

In [82]:
print_example()

Es läuft der Frühlingswind
Durch kahle Alleen,
Seltsame Dinge sind
In seinem jung geschah;
Empor, und wo mein Auge dich zu klagen, und mein Mund dir süßer ist,
So komm nur wieder zu sichen:
Die Sonnenländern
Die jeden tiefen Schmerz erquicklich macht
Und schmackhaft auch mein Ich im Wirbel, und war ich gern --
Nicht die Schwere vieler Erden,
Nu


In [83]:
model.save_weights('data/char_rnn_Hofmann.h5')